In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.essence import Shade, Essence
from everest.ptolemaic.ousia import Sprite, Ousia
from everest.epitaph import Epitaph, entomb, TAPHONOMY

In [ ]:
myobj = Sprite()

In [ ]:
Sprite.hashID

In [ ]:
myobj.hashID

In [ ]:
myobj.epitaph.decode()

In [ ]:
myobj.epitaph.hashID

In [ ]:
myobj._ptolemaic_class__

In [ ]:
type(myobj).__mro__[0].__bases__

In [ ]:
type(myobj).__mro__

In [ ]:
type(myobj).__mro__[1].__bases__

In [ ]:
type(myobj).get_epitaph

In [ ]:
myobj._ptolemaic_class__

In [ ]:
Sprite().epitaph

In [ ]:
TAPHONOMY.decode(Shade.epitaph)

In [ ]:
TAPHONOMY(Shade.epitaph).decode()

In [ ]:
class MyClass:

    def b(self):
        ...

    def a(self):
        ...

In [ ]:
MyClass.__dict__

In [ ]:
MyClass().__dict__

In [ ]:
dir(MyClass)

In [ ]:
dir(TAPHONOMY)

In [ ]:
TAPHONOMY.encoders

In [ ]:
Shade.epitaph

In [ ]:
TAPHONOMY.encode(Shade.epitaph)

In [ ]:
entomb(Shade.epitaph)

In [ ]:
type(type(Shade)).Epitaph

In [ ]:
type(Shade)

In [ ]:
from everest.epitaph import entomb

In [ ]:
epitaph = entomb((1, 'asdf', 2, entomb))
display(epitaph)
display(epitaph.decode())

In [ ]:
epitaph

In [ ]:
epitaph.decode()

In [ ]:
str(epitaph)

In [ ]:
Epitaph.

In [ ]:
mystr = """'abcd',<:<:'efgh'>,round,<:'ijkl'>>,<:<:'mnop'>,<c:'Primitive','everest.primitive'>,<:'qrst'>>,'uvwx','yz'"""

In [ ]:
levelpairs = tuple(TAPHONOMY.unpack_fences(mystr))
levelpairs

In [ ]:
tuple(TAPHONOMY.recursive_decode({}, levelpairs))

In [ ]:
TAPHONOMY.decode(mystr)

In [ ]:
_01

In [ ]:
from collections import abc as _collabc

In [ ]:
_collabc.Callable

In [ ]:
tup = tuple(TAPHONOMY.recursive_deco

In [ ]:
from everest.epitaph import entomb
from everest.primitive import Primitive

In [ ]:
myepi = entomb((1, 'asdf', 2, _Primitive))

In [ ]:
myepi

In [ ]:
myepi.decode()